In [ ]:
import numpy as np
import plotly.graph_objects as go

from plugin.managers.data_manager import DataManager
from spectrumlab.peaks.analyte_peaks.intensity.transformers import (
    RegressionIntensityTransformer,
    estimate_bounds,
    process_frame,
)


In [ ]:
data_manager = DataManager()
atom_data = data_manager.parse()

columns = list(atom_data.data.keys())
for i, column_id in enumerate(columns):
    print(i, column_id, atom_data.data[column_id].nickname)


In [ ]:
columns = list(atom_data.data.keys())

column_id = columns[0]
frame = atom_data.data[column_id].frame
bounds = atom_data.data[column_id].bounds
nickname = atom_data.data[column_id].nickname

print(nickname)

In [ ]:
data = process_frame(frame)

transformer = RegressionIntensityTransformer.create(
    data=data,
    bounds=[.01, .175],
    # bounds=estimate_bounds(data),
)
transformer.show()


In [ ]:
fig = go.Figure()

for probe_name, parallel_name in frame.index:
    if np.isnan(frame.loc[probe_name, parallel_name]['concentration']):
        continue

    y = transformer.apply(frame.loc[probe_name, parallel_name]['value']) / frame.loc[probe_name, parallel_name]['concentration']
    fig.add_trace(go.Scatter(
        y=y,
        mode='lines',
        name='{}/{} - {:.4f}'.format(probe_name, parallel_name, np.sum(transformer.apply(frame.loc[probe_name, parallel_name]['value']) / frame.loc[probe_name, parallel_name]['concentration'])),
        showlegend=True,
    ))

fig.update_layout(
    title=nickname,
    xaxis_title='time',
    yaxis_title='A',
    width=1800,
    height=800,
)

fig.show()